In [1]:
import pandas as pd
import numpy as np
import openpyxl
import warnings
warnings.filterwarnings('ignore')

rawdata = pd.read_excel('data/분석데이터_new.xlsx', engine='openpyxl')
rawdata.head()

,년/월,시도,산업_대분류,구인인원,취업건수,사업체 수,실업급여 수급자 수,각 지역별 인구,GDP(실질),금리,CLI 지수,소비자 심리지수
0,201301,서울,제조업,60,0,30219,97,10195064,1562.7,2.75,99.14964,101.0
1,201301,서울,도매 및 소매업,104,0,109382,202,10195064,1562.7,2.75,99.14964,101.0
2,201301,서울,정보통신업,763,4,23384,1230,10195064,1562.7,2.75,99.14964,101.0
3,201301,서울,"전문, 과학 및 기술 서비스업",35,1,33649,70,10195064,1562.7,2.75,99.14964,101.0
4,201301,서울,"사업시설 관리, 사업 지원 및 임대 서비스업",64,0,11578,62,10195064,1562.7,2.75,99.14964,101.0


In [2]:
rawdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10200 entries, 0 to 10199
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   년/월         10200 non-null  int64  
 1   시도          10200 non-null  object 
 2   산업_대분류      10200 non-null  object 
 3   구인인원        10200 non-null  int64  
 4   취업건수        10200 non-null  int64  
 5   사업체 수       10200 non-null  int64  
 6   실업급여 수급자 수  10200 non-null  int64  
 7   각 지역별 인구    10200 non-null  int64  
 8   GDP(실질)     10200 non-null  float64
 9   금리          10200 non-null  float64
 10  CLI 지수      10200 non-null  float64
 11  소비자 심리지수    10200 non-null  float64
dtypes: float64(4), int64(6), object(2)
memory usage: 956.4+ KB


In [3]:
rawdata.columns=['date', 'city', 'industry', 'job_offer', 'employment',
              'no_company', 'unemployment', 'population', 'GDP', 'i_rate', 'CLI', 'CFI']

In [4]:
rawdata['year'] = rawdata['date'].astype('str').str[:4].astype('int')
rawdata['month'] = rawdata['date'].astype('str').str[4:].astype('int')
rawdata.drop('date', axis=1, inplace=True)

In [5]:
print(len(rawdata) * 0.8)

8160.0


In [6]:
data1 = rawdata.copy()
train_data1 = data1.iloc[:8160, :] # 2013 ~ 2020년 데이터
test_data1 = data1.iloc[8160:, :] # 2021 ~ 2022년 데이터
display(train_data1.head(), test_data1.head())

,city,industry,job_offer,employment,no_company,unemployment,population,GDP,i_rate,CLI,CFI,year,month
0,서울,제조업,60,0,30219,97,10195064,1562.7,2.75,99.14964,101.0,2013,1
1,서울,도매 및 소매업,104,0,109382,202,10195064,1562.7,2.75,99.14964,101.0,2013,1
2,서울,정보통신업,763,4,23384,1230,10195064,1562.7,2.75,99.14964,101.0,2013,1
3,서울,"전문, 과학 및 기술 서비스업",35,1,33649,70,10195064,1562.7,2.75,99.14964,101.0,2013,1
4,서울,"사업시설 관리, 사업 지원 및 임대 서비스업",64,0,11578,62,10195064,1562.7,2.75,99.14964,101.0,2013,1


,city,industry,job_offer,employment,no_company,unemployment,population,GDP,i_rate,CLI,CFI,year,month
8160,서울,제조업,35,2,32388,115,9657969,1918.7,0.5,102.0237,95.4,2021,1
8161,서울,도매 및 소매업,34,0,131425,167,9657969,1918.7,0.5,102.0237,95.4,2021,1
8162,서울,정보통신업,633,7,32604,1431,9657969,1918.7,0.5,102.0237,95.4,2021,1
8163,서울,"전문, 과학 및 기술 서비스업",34,1,48282,122,9657969,1918.7,0.5,102.0237,95.4,2021,1
8164,서울,"사업시설 관리, 사업 지원 및 임대 서비스업",23,1,18711,104,9657969,1918.7,0.5,102.0237,95.4,2021,1


In [7]:
# 범주형 데이터 인코딩
from sklearn.preprocessing import LabelEncoder

object_features = ['city', 'industry']

for feature in object_features:
    le = LabelEncoder()
    le = le.fit(train_data1[feature])
    train_data1[feature] = le.transform(train_data1[feature])
    
    for label in np.unique(test_data1[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test_data1[feature]= le.transform(test_data1[feature])

In [8]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# MinMaxScaler를 이용한 수치 데이터 스케일링
scale_columns = ['employment', 'no_company', 'unemployment', 'population', 'GDP', 'i_rate', 'CLI', 'CFI']
scaler = MinMaxScaler()
scaler.fit(train_data1[scale_columns])
train_data1[scale_columns] = scaler.transform(train_data1[scale_columns])
test_data1[scale_columns] = scaler.transform(test_data1[scale_columns])

train_data1.head()

,city,industry,job_offer,employment,no_company,unemployment,population,GDP,i_rate,CLI,CFI,year,month
0,8,4,60,0.000000,0.227176,0.052291,0.757235,0.0,1.0,0.185561,0.715762,2013,1
1,8,0,104,0.000000,0.822790,0.108895,0.757235,0.0,1.0,0.185561,0.715762,2013,1
2,8,3,763,0.148148,0.175751,0.663073,0.757235,0.0,1.0,0.185561,0.715762,2013,1
3,8,2,35,0.037037,0.252983,0.037736,0.757235,0.0,1.0,0.185561,0.715762,2013,1
4,8,1,64,0.000000,0.086923,0.033423,0.757235,0.0,1.0,0.185561,0.715762,2013,1


In [9]:
from pycaret.regression import *
setup_rgs = setup(data=train_data1, target='job_offer', test_data=test_data1, session_id=42, preprocess=False)

,Description,Value
0,Session id,42
1,Target,job_offer
2,Target type,Regression
3,Original data shape,"(10200, 13)"
4,Transformed data shape,"(10200, 13)"
5,Transformed train set shape,"(8160, 13)"
6,Transformed test set shape,"(2040, 13)"
7,Numeric features,12


In [10]:
models()

,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model._base.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model._coordinate_descent.Lasso,True
ridge,Ridge Regression,sklearn.linear_model._ridge.Ridge,True
en,Elastic Net,sklearn.linear_model._coordinate_descent.Elast...,True
lar,Least Angle Regression,sklearn.linear_model._least_angle.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model._least_angle.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model._omp.OrthogonalMatchingPu...,True
br,Bayesian Ridge,sklearn.linear_model._bayes.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model._bayes.ARDRegression,False


In [11]:
top5_1 = compare_models(n_select=5, sort='MSE', turbo=False)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,09:02:17
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Linear Regression


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,6.0699,291.7292,16.5595,0.9014,0.6879,0.7353,0.6630
rf,Random Forest Regressor,6.1024,301.7456,16.8396,0.8974,0.6926,0.7550,1.3670
gbr,Gradient Boosting Regressor,6.5887,311.1730,17.2327,0.8917,0.8141,0.8357,0.3670
catboost,CatBoost Regressor,6.1672,327.0606,17.4950,0.8893,0.7193,0.7122,1.8580
xgboost,Extreme Gradient Boosting,6.6329,361.0931,18.3947,0.8788,0.7288,0.7580,0.0930
lightgbm,Light Gradient Boosting Machine,6.3425,402.8372,18.8957,0.8712,0.7149,0.7029,34.6460
knn,K Neighbors Regressor,7.4213,567.1086,21.5744,0.8378,0.8187,0.8670,0.0180
dt,Decision Tree Regressor,8.2859,627.7256,23.8900,0.8016,0.8607,1.0032,0.0280
kr,Kernel Ridge,10.6579,815.7205,27.0634,0.7523,1.1144,1.4979,2.1500
ridge,Ridge Regression,10.7128,816.0771,27.0719,0.7521,1.1209,1.5113,0.0080


In [12]:
top5_1

[ExtraTreesRegressor(n_jobs=-1, random_state=42),
 RandomForestRegressor(n_jobs=-1, random_state=42),
 GradientBoostingRegressor(random_state=42),
 XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=-1,
              num_parallel_tree=None, random_state=42, ...)]

In [13]:
tuned_top5_1 = [tune_model(i) for i in top5_1]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,9.9300,992.4967,31.5039,0.8592,0.8752,0.9013
1,7.2932,363.3333,19.0613,0.9224,0.8428,0.8520
2,7.1120,324.1488,18.0041,0.8736,0.9023,0.8536
3,5.5071,144.2498,12.0104,0.9528,0.8928,0.7687
4,5.5288,117.9310,10.8596,0.9533,0.9326,0.7220
5,5.6691,146.6295,12.1091,0.9366,0.8199,0.7042
6,5.6784,293.6030,17.1348,0.8357,0.7520,0.7833
7,5.1468,238.9542,15.4581,0.8993,0.7805,0.9106
8,5.4832,228.9183,15.1300,0.8954,0.7781,0.7864


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,9.9275,957.6175,30.9454,0.8641,0.8693,0.8270
1,7.2516,384.9740,19.6208,0.9178,0.8394,0.7563
2,6.9128,307.8385,17.5453,0.8799,0.9165,0.7503
3,5.5718,178.4224,13.3575,0.9417,0.9196,0.6669
4,5.6551,114.3991,10.6957,0.9547,0.9682,0.6756
5,5.8417,144.5776,12.0240,0.9375,0.8438,0.6429
6,6.0201,302.3840,17.3892,0.8308,0.8163,0.8207
7,5.5918,296.7214,17.2256,0.8749,0.8486,0.9790
8,5.8484,253.3919,15.9183,0.8842,0.8491,0.8941


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,9.4817,905.2052,30.0866,0.8715,0.7411,0.7781
1,6.8974,372.8569,19.3095,0.9204,0.7517,0.7194
2,6.7451,342.6540,18.5109,0.8664,0.7851,0.7174
3,5.0419,156.1096,12.4944,0.9490,0.7065,0.6154
4,5.4473,161.6850,12.7155,0.9359,0.7650,0.6894
5,6.2616,158.7599,12.6000,0.9314,0.7973,0.7957
6,6.0844,305.4234,17.4764,0.8291,0.7042,0.7858
7,4.8069,273.6305,16.5418,0.8847,0.6479,0.7167
8,5.1381,259.9630,16.1234,0.8812,0.6266,0.6466


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,9.0153,619.9994,24.8998,0.9120,0.7674,0.8566
1,8.4409,409.5821,20.2381,0.9126,0.8855,0.9287
2,7.3344,408.8442,20.2199,0.8406,0.9037,0.8050
3,5.5890,173.8392,13.1848,0.9432,0.8614,0.6708
4,6.5638,204.3403,14.2948,0.9190,0.9984,1.0023
5,6.8499,172.0731,13.1177,0.9257,1.0010,1.0641
6,6.4094,312.2796,17.6714,0.8253,0.8007,0.9505
7,5.8488,286.4374,16.9245,0.8793,0.8321,1.0065
8,5.8783,311.5500,17.6508,0.8576,0.6944,0.7538


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,10.9456,1328.6770,36.4510,0.8115,0.7596,0.7974
1,7.1281,350.7852,18.7293,0.9251,0.7753,0.7458
2,6.6918,408.9677,20.2229,0.8405,0.7352,0.5454
3,5.3278,192.2506,13.8654,0.9371,0.7304,0.5882
4,5.3291,147.6866,12.1526,0.9415,0.7522,0.6120
5,6.0617,176.3538,13.2798,0.9238,0.7880,0.7150
6,5.8192,230.8419,15.1935,0.8708,0.6983,0.7131
7,5.0185,275.2406,16.5904,0.8840,0.6535,0.6110
8,5.4342,253.9780,15.9367,0.8839,0.6199,0.5962


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [14]:
# 혼합 모델 생성
blender_5_1 = blend_models(tuned_top5_1)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,8.3022,482.9345,21.9758,0.9315,0.7197,0.7617
1,6.8449,362.5586,19.0410,0.9226,0.7194,0.6936
2,6.6789,364.9177,19.1028,0.8577,0.7530,0.7272
3,4.7732,135.9986,11.6618,0.9555,0.6679,0.6147
4,4.9696,111.7526,10.5713,0.9557,0.7482,0.7114
5,5.6361,136.5412,11.6851,0.9410,0.7917,0.7457
6,5.7840,305.7293,17.4851,0.8289,0.6598,0.7549
7,4.6066,211.8077,14.5536,0.9107,0.6043,0.6658
8,5.0857,245.2489,15.6604,0.8879,0.6333,0.6693


In [15]:
result1 = pull()
display(result1)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,8.3022,482.9345,21.9758,0.9315,0.7197,0.7617
1,6.8449,362.5586,19.0410,0.9226,0.7194,0.6936
2,6.6789,364.9177,19.1028,0.8577,0.7530,0.7272
3,4.7732,135.9986,11.6618,0.9555,0.6679,0.6147
4,4.9696,111.7526,10.5713,0.9557,0.7482,0.7114
5,5.6361,136.5412,11.6851,0.9410,0.7917,0.7457
6,5.7840,305.7293,17.4851,0.8289,0.6598,0.7549
7,4.6066,211.8077,14.5536,0.9107,0.6043,0.6658
8,5.0857,245.2489,15.6604,0.8879,0.6333,0.6693


In [16]:
final_model1 = finalize_model(blender_5_1)
prediction1 = predict_model(final_model1, data=test_data1.drop('job_offer', axis=1))
prediction1

,city,industry,employment,no_company,unemployment,population,GDP,i_rate,CLI,CFI,year,month,prediction_label
8160,8,4,0.074074,0.243496,0.061995,0.716892,1.227586,0.000000,1.117550,0.571059,2021,1,37.720505
8161,8,0,0.000000,0.988639,0.090027,0.716892,1.227586,0.000000,1.117550,0.571059,2021,1,41.442569
8162,8,3,0.259259,0.245121,0.771429,0.716892,1.227586,0.000000,1.117550,0.571059,2021,1,608.972100
8163,8,2,0.037037,0.363080,0.065768,0.716892,1.227586,0.000000,1.117550,0.571059,2021,1,37.766810
8164,8,1,0.037037,0.140591,0.056065,0.716892,1.227586,0.000000,1.117550,0.571059,2021,1,22.945079
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10195,14,4,0.000000,0.015868,0.001078,0.042383,1.400345,1.222222,0.056255,0.436693,2022,12,2.276023
10196,14,0,0.000000,0.055376,0.003235,0.042383,1.400345,1.222222,0.056255,0.436693,2022,12,0.409212
10197,14,3,0.000000,0.004018,0.012399,0.042383,1.400345,1.222222,0.056255,0.436693,2022,12,9.192957
10198,14,2,0.000000,0.011557,0.002156,0.042383,1.400345,1.222222,0.056255,0.436693,2022,12,1.210386


In [17]:
# save_model(model=final_model1, 
#            model_name='final_rf',
#            verbose=False)

In [18]:
real_pred = pd.DataFrame({'real':test_data1['job_offer'], 'pred':prediction1['prediction_label']})
real_pred['pred'] = real_pred['pred'].astype('int')
real_pred

,real,pred
8160,35,37
8161,34,41
8162,633,608
8163,34,37
8164,23,22
...,...,...
10195,0,2
10196,0,0
10197,8,9
10198,1,1


In [19]:
real_pred.loc[real_pred['real'] != real_pred['pred']]['pred'].value_counts()

1      211
2      127
3      105
4       68
5       66
      ... 
252      1
69       1
768      1
217      1
593      1
Name: pred, Length: 131, dtype: int64

In [67]:
print('정답률: ', 100-((131/2040)*100), '%', sep='') # 원래 126. 정답률은 조금 낮아짐

정답률: 93.57843137254902%


- 2번째
    - One-Hot Encoding

In [21]:
data2 = rawdata.copy()
train_data2 = data2.iloc[:8160, :].reset_index(drop=True) # 2013 ~ 2020년 데이터
test_data2 = data2.iloc[8160:, :].reset_index(drop=True) # 2021 ~ 2022년 데이터
print(len(train_data2), len(test_data2))

8160 2040


In [22]:
# One-Hot encoding
from sklearn.preprocessing import OneHotEncoder

object_features = ['city', 'industry']

for feature in object_features:
    oe = OneHotEncoder(sparse=False, drop='first')
    oe.fit(train_data2[[feature]])
    train_cat = oe.transform(train_data2[[feature]])
    train_cats = pd.DataFrame(train_cat, columns=[col for col in oe.categories_[0][1:]])
    train_data2 = pd.concat([train_data2, train_cats], axis=1)
    
    for label in np.unique(test_data2[feature]):
        if label not in list(oe.categories_[0]):
            oe.categories_ = np.append(oe.categories_, label)
    test_cat = oe.transform(test_data2[[feature]])
    test_cats = pd.DataFrame(test_cat, columns=[col for col in oe.categories_[0][1:]])
    test_data2 = pd.concat([test_data2, test_cats], axis=1)
    
    train_data2.drop(feature, axis=1, inplace=True)
    test_data2.drop(feature, axis=1, inplace=True)

In [23]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# MinMaxScaler를 이용한 수치 데이터 스케일링
scale_columns = ['employment', 'no_company', 'unemployment', 'population', 'GDP', 'i_rate', 'CLI', 'CFI']
scaler = MinMaxScaler()
scaler.fit(train_data2[scale_columns])
train_data2[scale_columns] = scaler.transform(train_data2[scale_columns])
test_data2[scale_columns] = scaler.transform(test_data2[scale_columns])

train_data2.head()

,job_offer,employment,no_company,unemployment,population,GDP,i_rate,CLI,CFI,year,...,인천,전남,전북,제주,충남,충북,"사업시설 관리, 사업 지원 및 임대 서비스업","전문, 과학 및 기술 서비스업",정보통신업,제조업
0,60,0.000000,0.227176,0.052291,0.757235,0.0,1.0,0.185561,0.715762,2013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,104,0.000000,0.822790,0.108895,0.757235,0.0,1.0,0.185561,0.715762,2013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,763,0.148148,0.175751,0.663073,0.757235,0.0,1.0,0.185561,0.715762,2013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,35,0.037037,0.252983,0.037736,0.757235,0.0,1.0,0.185561,0.715762,2013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,64,0.000000,0.086923,0.033423,0.757235,0.0,1.0,0.185561,0.715762,2013,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [24]:
test_data2.head()

,job_offer,employment,no_company,unemployment,population,GDP,i_rate,CLI,CFI,year,...,인천,전남,전북,제주,충남,충북,"사업시설 관리, 사업 지원 및 임대 서비스업","전문, 과학 및 기술 서비스업",정보통신업,제조업
0,35,0.074074,0.243496,0.061995,0.716892,1.227586,0.0,1.11755,0.571059,2021,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,34,0.000000,0.988639,0.090027,0.716892,1.227586,0.0,1.11755,0.571059,2021,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,633,0.259259,0.245121,0.771429,0.716892,1.227586,0.0,1.11755,0.571059,2021,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,34,0.037037,0.363080,0.065768,0.716892,1.227586,0.0,1.11755,0.571059,2021,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,23,0.037037,0.140591,0.056065,0.716892,1.227586,0.0,1.11755,0.571059,2021,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [25]:
from pycaret.regression import *
setup_rgs = setup(data=train_data2, target='job_offer', test_data=test_data2, session_id=42, index=False, preprocess=False)

,Description,Value
0,Session id,42
1,Target,job_offer
2,Target type,Regression
3,Original data shape,"(10200, 31)"
4,Transformed data shape,"(10200, 31)"
5,Transformed train set shape,"(8160, 31)"
6,Transformed test set shape,"(2040, 31)"
7,Numeric features,30


In [26]:
top5_2 = compare_models(n_select=5, sort='MSE', turbo=False)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,09:23:01
Status,. . . . . . . . . . . . . . . . . .,Initializing CV
Estimator,. . . . . . . . . . . . . . . . . .,Linear Regression


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,6.0068,287.1064,16.4179,0.9036,0.6741,0.7207,0.9030
rf,Random Forest Regressor,5.9857,297.4298,16.7417,0.8982,0.6799,0.7329,1.6480
gbr,Gradient Boosting Regressor,6.5332,308.5148,17.1435,0.8929,0.8050,0.8127,0.4240
catboost,CatBoost Regressor,6.2217,339.0939,17.7998,0.8878,0.7242,0.7237,1.6560
lightgbm,Light Gradient Boosting Machine,6.3527,401.1916,18.8389,0.8731,0.7095,0.7038,36.3030
xgboost,Extreme Gradient Boosting,6.7971,442.4744,19.6207,0.8513,0.7175,0.7421,0.0980
dt,Decision Tree Regressor,7.7779,540.3404,22.0849,0.8285,0.8490,0.9382,0.0320
knn,K Neighbors Regressor,6.9537,543.2390,21.0350,0.8447,0.7377,0.7152,0.0300
kr,Kernel Ridge,10.7404,741.0221,25.8139,0.7745,1.1046,1.5866,1.9200
ridge,Ridge Regression,10.8265,741.6104,25.8263,0.7743,1.1106,1.6226,0.0110


In [27]:
top5_2

[ExtraTreesRegressor(n_jobs=-1, random_state=42),
 RandomForestRegressor(n_jobs=-1, random_state=42),
 GradientBoostingRegressor(random_state=42),
 LGBMRegressor(n_jobs=-1, random_state=42)]

In [29]:
tuned_top5_2 = [tune_model(i) for i in top5_2]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,10.3161,827.5208,28.7667,0.8826,0.8945,0.7506
1,7.5798,369.8219,19.2308,0.9211,0.8537,0.7131
2,7.4459,378.8192,19.4633,0.8523,0.9711,0.7499
3,6.1534,173.8228,13.1842,0.9432,1.0190,0.7351
4,5.9411,107.5595,10.3711,0.9574,1.0544,0.7363
5,6.0240,142.9163,11.9548,0.9383,0.9305,0.7194
6,6.4015,317.2041,17.8102,0.8225,0.8774,0.9519
7,5.9915,238.0745,15.4297,0.8997,0.9098,1.1281
8,6.4485,242.4085,15.5695,0.8892,0.9279,1.0964


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,10.1322,951.1191,30.8402,0.8650,0.8883,0.8401
1,7.3928,363.7006,19.0709,0.9224,0.8728,0.8075
2,7.1414,315.1273,17.7518,0.8771,0.9602,0.8357
3,5.9183,189.9340,13.7817,0.9379,0.9850,0.7548
4,5.7890,108.7701,10.4293,0.9569,1.0118,0.7505
5,6.1425,148.0792,12.1688,0.9360,0.8973,0.7192
6,6.2248,310.5190,17.6216,0.8263,0.8675,0.9011
7,5.8066,247.2282,15.7235,0.8958,0.9081,1.0988
8,6.2256,242.4728,15.5715,0.8892,0.9151,1.0348


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,9.4870,1072.8181,32.7539,0.8478,0.7442,0.7427
1,6.9105,351.6296,18.7518,0.9249,0.7357,0.7205
2,6.6416,311.3341,17.6447,0.8786,0.7668,0.7591
3,5.0662,162.0837,12.7312,0.9470,0.7107,0.6478
4,5.4252,130.8635,11.4396,0.9481,0.8223,0.7645
5,5.9620,146.8557,12.1184,0.9366,0.8244,0.8047
6,6.1225,381.7273,19.5378,0.7864,0.7184,0.7987
7,4.7397,259.6797,16.1146,0.8906,0.6601,0.7701
8,5.3735,261.8976,16.1832,0.8803,0.6655,0.7188


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,9.1370,735.7421,27.1246,0.8956,0.8288,0.8939
1,7.7908,366.8810,19.1541,0.9217,0.8946,0.7940
2,6.9603,339.6685,18.4301,0.8675,0.8584,0.7339
3,5.5862,194.2875,13.9387,0.9365,0.7931,0.6104
4,6.2713,176.2970,13.2777,0.9301,0.9508,0.7904
5,7.5345,214.4237,14.6432,0.9074,1.0119,1.1705
6,6.3745,333.6811,18.2669,0.8133,0.7606,0.8991
7,5.5752,316.0010,17.7764,0.8668,0.7457,0.9041
8,6.0964,283.4547,16.8361,0.8705,0.7591,0.8896


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,10.2619,952.4772,30.8622,0.8648,0.8421,0.9199
1,7.7397,387.3609,19.6815,0.9173,0.8783,0.7799
2,7.5321,389.0547,19.7245,0.8483,0.8541,0.8942
3,5.8719,179.7025,13.4053,0.9412,0.8080,0.7417
4,6.3995,182.2922,13.5016,0.9278,0.8358,0.8227
5,6.8634,159.9625,12.6476,0.9309,0.9374,0.9556
6,6.1725,252.1045,15.8778,0.8590,0.7815,0.9069
7,5.7516,295.7394,17.1971,0.8754,0.7281,0.8289
8,6.7970,327.2334,18.0896,0.8505,0.7413,1.0479


Fitting 10 folds for each of 10 candidates, totalling 100 fits
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5[LightGBM] [Warning] fea

In [30]:
tuned_top5_2

[ExtraTreesRegressor(n_jobs=-1, random_state=42),
 RandomForestRegressor(n_jobs=-1, random_state=42),
 GradientBoostingRegressor(random_state=42),
 LGBMRegressor(bagging_fraction=0.8, bagging_freq=3, feature_fraction=1.0,
               learning_rate=0.5, min_child_samples=100, min_split_gain=0.3,
               n_estimators=190, n_jobs=-1, num_leaves=10, random_state=42,
               reg_alpha=0.1, reg_lambda=0.05)]

In [31]:
# 혼합 모델 생성
blender_5_2 = blend_models(tuned_top5_2)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,8.3321,503.7691,22.4448,0.9285,0.7464,0.7990
1,6.6094,346.4088,18.6121,0.9261,0.7003,0.6417
2,6.6456,348.2840,18.6624,0.8642,0.7367,0.7419
3,4.7600,143.8959,11.9957,0.9530,0.6495,0.6107
4,5.0084,106.1428,10.3026,0.9579,0.7462,0.6826
5,5.4629,122.9461,11.0881,0.9469,0.8064,0.7418
6,5.4455,265.5148,16.2946,0.8514,0.6601,0.7138
7,4.7297,228.0651,15.1018,0.9039,0.6184,0.6708
8,5.1921,259.6348,16.1132,0.8813,0.6267,0.6924


[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000714 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=tru

In [32]:
result2 = pull()
display(result2)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,8.3321,503.7691,22.4448,0.9285,0.7464,0.7990
1,6.6094,346.4088,18.6121,0.9261,0.7003,0.6417
2,6.6456,348.2840,18.6624,0.8642,0.7367,0.7419
3,4.7600,143.8959,11.9957,0.9530,0.6495,0.6107
4,5.0084,106.1428,10.3026,0.9579,0.7462,0.6826
5,5.4629,122.9461,11.0881,0.9469,0.8064,0.7418
6,5.4455,265.5148,16.2946,0.8514,0.6601,0.7138
7,4.7297,228.0651,15.1018,0.9039,0.6184,0.6708
8,5.1921,259.6348,16.1132,0.8813,0.6267,0.6924


In [33]:
final_model2 = finalize_model(blender_5_2)
prediction2 = predict_model(final_model2, data=test_data2.drop('job_offer', axis=1))
prediction2

[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.126834 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1079
[LightGBM] [Info] Number o

[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8


,employment,no_company,unemployment,population,GDP,i_rate,CLI,CFI,year,month,...,전남,전북,제주,충남,충북,"사업시설 관리, 사업 지원 및 임대 서비스업","전문, 과학 및 기술 서비스업",정보통신업,제조업,prediction_label
0,0.074074,0.243496,0.061995,0.716892,1.227586,0.000000,1.117550,0.571059,2021,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,38.701853
1,0.000000,0.988639,0.090027,0.716892,1.227586,0.000000,1.117550,0.571059,2021,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.765138
2,0.259259,0.245121,0.771429,0.716892,1.227586,0.000000,1.117550,0.571059,2021,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,578.265065
3,0.037037,0.363080,0.065768,0.716892,1.227586,0.000000,1.117550,0.571059,2021,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,37.717329
4,0.037037,0.140591,0.056065,0.716892,1.227586,0.000000,1.117550,0.571059,2021,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,21.395325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2035,0.000000,0.015868,0.001078,0.042383,1.400345,1.222222,0.056255,0.436693,2022,12,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.762231
2036,0.000000,0.055376,0.003235,0.042383,1.400345,1.222222,0.056255,0.436693,2022,12,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.636563
2037,0.000000,0.004018,0.012399,0.042383,1.400345,1.222222,0.056255,0.436693,2022,12,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,8.654370
2038,0.000000,0.011557,0.002156,0.042383,1.400345,1.222222,0.056255,0.436693,2022,12,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.899797


In [34]:
real_pred2 = pd.DataFrame({'real':test_data2['job_offer'], 'pred':prediction2['prediction_label']})
real_pred2['pred'] = real_pred2['pred'].astype('int')
real_pred2

,real,pred
0,35,38
1,34,45
2,633,578
3,34,37
4,23,21
...,...,...
2035,0,1
2036,0,0
2037,8,8
2038,1,0


In [35]:
real_pred2.loc[real_pred2['real'] != real_pred2['pred']]['pred'].value_counts()

1      186
2      160
3      109
4       76
5       72
      ... 
763      1
210      1
643      1
110      1
184      1
Name: pred, Length: 129, dtype: int64

In [68]:
print('정답률: ', 100-((129/2040)*100), '%', sep='')

정답률: 93.67647058823529%


- labelencoding보다 one-hot encoding이 결과가 더 좋게 나옴

- 데이터에 이상치가 다수 존재. robustscaler를 이용해보기

In [37]:
data3 = rawdata.copy()
train_data3 = data3.iloc[:8160, :].reset_index(drop=True) # 2013 ~ 2020년 데이터
test_data3 = data3.iloc[8160:, :].reset_index(drop=True) # 2021 ~ 2022년 데이터
print(len(train_data3), len(test_data3))

8160 2040


In [38]:
# One-Hot encoding
from sklearn.preprocessing import OneHotEncoder

object_features = ['city', 'industry']

for feature in object_features:
    oe = OneHotEncoder(sparse=False, drop='first')
    oe.fit(train_data3[[feature]])
    train_cat = oe.transform(train_data3[[feature]])
    train_cats = pd.DataFrame(train_cat, columns=[col for col in oe.categories_[0][1:]])
    train_data3 = pd.concat([train_data3, train_cats], axis=1)
    
    for label in np.unique(test_data3[feature]):
        if label not in list(oe.categories_[0]):
            oe.categories_ = np.append(oe.categories_, label)
    test_cat = oe.transform(test_data3[[feature]])
    test_cats = pd.DataFrame(test_cat, columns=[col for col in oe.categories_[0][1:]])
    test_data3 = pd.concat([test_data3, test_cats], axis=1)
    
    train_data3.drop(feature, axis=1, inplace=True)
    test_data3.drop(feature, axis=1, inplace=True)

In [39]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# MinMaxScaler를 이용한 수치 데이터 스케일링
scale_columns = ['employment', 'no_company', 'unemployment', 'population', 'GDP', 'i_rate', 'CLI', 'CFI']
scaler = RobustScaler()
scaler.fit(train_data3[scale_columns])
train_data3[scale_columns] = scaler.transform(train_data3[scale_columns])
test_data3[scale_columns] = scaler.transform(test_data3[scale_columns])

train_data3.head()

,job_offer,employment,no_company,unemployment,population,GDP,i_rate,CLI,CFI,year,...,인천,전남,전북,제주,충남,충북,"사업시설 관리, 사업 지원 및 임대 서비스업","전문, 과학 및 기술 서비스업",정보통신업,제조업
0,60,0.0,2.522520,3.869565,5.671572,-0.993902,1.666667,-0.683205,-0.090566,2013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,104,0.0,9.851410,8.434783,5.671572,-0.993902,1.666667,-0.683205,-0.090566,2013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,763,4.0,1.889738,53.130435,5.671572,-0.993902,1.666667,-0.683205,-0.090566,2013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,35,1.0,2.840069,2.695652,5.671572,-0.993902,1.666667,-0.683205,-0.090566,2013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,64,0.0,0.796741,2.347826,5.671572,-0.993902,1.666667,-0.683205,-0.090566,2013,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [40]:
test_data3.head()

,job_offer,employment,no_company,unemployment,population,GDP,i_rate,CLI,CFI,year,...,인천,전남,전북,제주,충남,충북,"사업시설 관리, 사업 지원 및 임대 서비스업","전문, 과학 및 기술 서비스업",정보통신업,제조업
0,35,2.0,2.723325,4.652174,5.304254,1.073461,-1.333333,2.07648,-0.935849,2021,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,34,0.0,11.892145,6.913043,5.304254,1.073461,-1.333333,2.07648,-0.935849,2021,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,633,7.0,2.743323,61.869565,5.304254,1.073461,-1.333333,2.07648,-0.935849,2021,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,34,1.0,4.194788,4.956522,5.304254,1.073461,-1.333333,2.07648,-0.935849,2021,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,23,1.0,1.457112,4.173913,5.304254,1.073461,-1.333333,2.07648,-0.935849,2021,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [41]:
from pycaret.regression import *
setup_rgs = setup(data=train_data3, target='job_offer', test_data=test_data3, session_id=42, index=False, preprocess=False)

,Description,Value
0,Session id,42
1,Target,job_offer
2,Target type,Regression
3,Original data shape,"(10200, 31)"
4,Transformed data shape,"(10200, 31)"
5,Transformed train set shape,"(8160, 31)"
6,Transformed test set shape,"(2040, 31)"
7,Numeric features,30


In [42]:
top5_3 = compare_models(n_select=5, sort='MSE', turbo=False)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:36:44
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Linear Regression


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,5.9962,284.4744,16.3432,0.9038,0.6776,0.7223,0.9270
rf,Random Forest Regressor,6.0034,297.0017,16.7159,0.8984,0.6795,0.7326,1.6190
gbr,Gradient Boosting Regressor,6.5294,308.1208,17.1332,0.8930,0.8049,0.8126,0.4290
catboost,CatBoost Regressor,6.2214,339.0894,17.7997,0.8878,0.7242,0.7237,1.8350
lightgbm,Light Gradient Boosting Machine,6.2813,383.4464,18.5542,0.8797,0.7042,0.6957,39.1760
xgboost,Extreme Gradient Boosting,6.7971,442.4744,19.6207,0.8513,0.7175,0.7421,0.1010
dt,Decision Tree Regressor,7.8809,534.1990,22.1457,0.8271,0.8546,0.9597,0.0330
knn,K Neighbors Regressor,7.5048,606.0257,23.2108,0.7976,0.7449,0.7659,0.0310
mlp,MLP Regressor,11.2967,717.5131,25.2596,0.7815,1.2100,1.8518,2.6810
kr,Kernel Ridge,10.8059,741.8707,25.8513,0.7721,1.1095,1.5995,2.0600


In [43]:
top5_3

[ExtraTreesRegressor(n_jobs=-1, random_state=42),
 RandomForestRegressor(n_jobs=-1, random_state=42),
 GradientBoostingRegressor(random_state=42),
 LGBMRegressor(n_jobs=-1, random_state=42)]

In [44]:
tuned_top5_3 = [tune_model(i) for i in top5_3]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,10.3161,827.5208,28.7667,0.8826,0.8945,0.7506
1,7.5798,369.8219,19.2308,0.9211,0.8537,0.7131
2,7.4459,378.8192,19.4633,0.8523,0.9711,0.7499
3,6.1534,173.8228,13.1842,0.9432,1.0190,0.7351
4,5.9411,107.5595,10.3711,0.9574,1.0544,0.7363
5,6.0240,142.9163,11.9548,0.9383,0.9305,0.7194
6,6.4015,317.2041,17.8102,0.8225,0.8774,0.9519
7,5.9915,238.0745,15.4297,0.8997,0.9098,1.1281
8,6.4485,242.4085,15.5695,0.8892,0.9279,1.0964


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,10.1303,951.3665,30.8442,0.8650,0.8883,0.8400
1,7.4020,364.1401,19.0825,0.9223,0.8728,0.8077
2,7.1457,316.1600,17.7809,0.8767,0.9602,0.8358
3,5.9177,189.7910,13.7765,0.9379,0.9849,0.7542
4,5.7886,108.7844,10.4300,0.9569,1.0119,0.7515
5,6.1476,148.7274,12.1954,0.9357,0.8963,0.7195
6,6.2279,310.9923,17.6350,0.8260,0.8672,0.9016
7,5.8159,248.4743,15.7631,0.8953,0.9075,1.0990
8,6.2259,242.4687,15.5714,0.8892,0.9147,1.0340


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,9.5312,1102.0927,33.1978,0.8436,0.7437,0.7424
1,6.9761,365.9567,19.1300,0.9219,0.7430,0.7392
2,6.4879,304.3526,17.4457,0.8813,0.7476,0.7296
3,4.9114,134.1978,11.5844,0.9561,0.7228,0.6454
4,5.4693,143.2237,11.9676,0.9432,0.8198,0.7626
5,6.0523,150.3739,12.2627,0.9350,0.8025,0.8160
6,6.0560,348.0817,18.6569,0.8053,0.7079,0.7935
7,4.7363,264.4799,16.2628,0.8885,0.6701,0.7722
8,5.2219,235.1105,15.3333,0.8926,0.6615,0.7176


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,9.1370,735.7421,27.1246,0.8956,0.8288,0.8939
1,7.7908,366.8810,19.1541,0.9217,0.8946,0.7940
2,6.9603,339.6685,18.4301,0.8675,0.8584,0.7339
3,5.5862,194.2875,13.9387,0.9365,0.7931,0.6104
4,6.2713,176.2970,13.2777,0.9301,0.9508,0.7904
5,7.5345,214.4237,14.6432,0.9074,1.0119,1.1705
6,6.3745,333.6811,18.2669,0.8133,0.7606,0.8991
7,5.5752,316.0010,17.7764,0.8668,0.7457,0.9041
8,6.0964,283.4547,16.8361,0.8705,0.7591,0.8896


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,10.3633,946.1207,30.7591,0.8657,0.8095,0.9150
1,8.2926,396.8271,19.9205,0.9153,0.9001,0.8346
2,7.7094,407.7559,20.1930,0.8410,0.8828,0.8280
3,5.5739,122.9331,11.0875,0.9598,0.8112,0.7585
4,5.9182,154.1626,12.4162,0.9389,0.8310,0.7041
5,6.7907,175.4089,13.2442,0.9242,0.8945,0.9068
6,6.4845,269.3767,16.4127,0.8493,0.8099,0.9494
7,5.6196,295.3911,17.1869,0.8755,0.7221,0.8311
8,6.5720,313.3940,17.7029,0.8568,0.7329,0.9157


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [45]:
tuned_top5_3

[ExtraTreesRegressor(n_jobs=-1, random_state=42),
 RandomForestRegressor(n_jobs=-1, random_state=42),
 GradientBoostingRegressor(random_state=42),
 LGBMRegressor(bagging_fraction=0.8, bagging_freq=3, feature_fraction=1.0,
               learning_rate=0.5, min_child_samples=100, min_split_gain=0.3,
               n_estimators=190, n_jobs=-1, num_leaves=10, random_state=42,
               reg_alpha=0.1, reg_lambda=0.05)]

In [46]:
# 혼합 모델 생성
blender_5_3 = blend_models(tuned_top5_3)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,8.3326,503.1959,22.4320,0.9286,0.7410,0.7956
1,6.6295,346.5354,18.6155,0.9260,0.7039,0.6274
2,6.6710,351.2952,18.7429,0.8630,0.7386,0.7272
3,4.7047,129.5569,11.3823,0.9576,0.6417,0.6241
4,4.9366,104.1188,10.2039,0.9587,0.7444,0.6530
5,5.5117,126.5564,11.2497,0.9453,0.8030,0.7447
6,5.5339,270.5425,16.4482,0.8486,0.6655,0.7366
7,4.6957,227.9617,15.0984,0.9039,0.6184,0.6771
8,5.2182,258.1181,16.0661,0.8820,0.6382,0.6805


In [47]:
result3 = pull()
display(result3)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,8.3326,503.1959,22.4320,0.9286,0.7410,0.7956
1,6.6295,346.5354,18.6155,0.9260,0.7039,0.6274
2,6.6710,351.2952,18.7429,0.8630,0.7386,0.7272
3,4.7047,129.5569,11.3823,0.9576,0.6417,0.6241
4,4.9366,104.1188,10.2039,0.9587,0.7444,0.6530
5,5.5117,126.5564,11.2497,0.9453,0.8030,0.7447
6,5.5339,270.5425,16.4482,0.8486,0.6655,0.7366
7,4.6957,227.9617,15.0984,0.9039,0.6184,0.6771
8,5.2182,258.1181,16.0661,0.8820,0.6382,0.6805


In [48]:
final_model3 = finalize_model(blender_5_3)
prediction3 = predict_model(final_model3, data=test_data3.drop('job_offer', axis=1))
prediction3

[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8


,employment,no_company,unemployment,population,GDP,i_rate,CLI,CFI,year,month,...,전남,전북,제주,충남,충북,"사업시설 관리, 사업 지원 및 임대 서비스업","전문, 과학 및 기술 서비스업",정보통신업,제조업,prediction_label
0,2.0,2.723325,4.652174,5.304254,1.073461,-1.333333,2.076480,-0.935849,2021,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,37.574499
1,0.0,11.892144,6.913043,5.304254,1.073461,-1.333333,2.076480,-0.935849,2021,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.517602
2,7.0,2.743323,61.869564,5.304254,1.073461,-1.333333,2.076480,-0.935849,2021,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,580.668956
3,1.0,4.194788,4.956522,5.304254,1.073461,-1.333333,2.076480,-0.935849,2021,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,38.398677
4,1.0,1.457112,4.173913,5.304254,1.073461,-1.333333,2.076480,-0.935849,2021,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,23.313675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2035,0.0,-0.077582,-0.260870,-0.837016,1.364402,2.333333,-1.066086,-1.720755,2022,12,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.930714
2036,0.0,0.408554,-0.086957,-0.837016,1.364402,2.333333,-1.066086,-1.720755,2022,12,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.462932
2037,0.0,-0.223395,0.652174,-0.837016,1.364402,2.333333,-1.066086,-1.720755,2022,12,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,8.796354
2038,0.0,-0.130630,-0.173913,-0.837016,1.364402,2.333333,-1.066086,-1.720755,2022,12,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.648587


In [49]:
real_pred3 = pd.DataFrame({'real':test_data3['job_offer'], 'pred':prediction3['prediction_label']})
real_pred3['pred'] = real_pred3['pred'].astype('int')
real_pred3

,real,pred
0,35,37
1,34,45
2,633,580
3,34,38
4,23,23
...,...,...
2035,0,1
2036,0,0
2037,8,8
2038,1,0


In [50]:
real_pred3.loc[real_pred3['real'] != real_pred3['pred']]['pred'].value_counts()

1      179
2      154
3      116
4       77
6       63
      ... 
206      1
644      1
111      1
148      1
184      1
Name: pred, Length: 135, dtype: int64

In [69]:
print('정답률: ', 100-((135/2040)*100), '%', sep='')

정답률: 93.38235294117646%


- 스케일링을 하지 않고 원본으로도 해보기

In [52]:
data4 = rawdata.copy()
train_data4 = data4.iloc[:8160, :].reset_index(drop=True) # 2013 ~ 2020년 데이터
test_data4 = data4.iloc[8160:, :].reset_index(drop=True) # 2021 ~ 2022년 데이터
print(len(train_data4), len(test_data4))

8160 2040


In [53]:
# One-Hot encoding
from sklearn.preprocessing import OneHotEncoder

object_features = ['city', 'industry']

for feature in object_features:
    oe = OneHotEncoder(sparse=False, drop='first')
    oe.fit(train_data4[[feature]])
    train_cat = oe.transform(train_data4[[feature]])
    train_cats = pd.DataFrame(train_cat, columns=[col for col in oe.categories_[0][1:]])
    train_data4 = pd.concat([train_data4, train_cats], axis=1)
    
    for label in np.unique(test_data4[feature]):
        if label not in list(oe.categories_[0]):
            oe.categories_ = np.append(oe.categories_, label)
    test_cat = oe.transform(test_data4[[feature]])
    test_cats = pd.DataFrame(test_cat, columns=[col for col in oe.categories_[0][1:]])
    test_data4 = pd.concat([test_data4, test_cats], axis=1)
    
    train_data4.drop(feature, axis=1, inplace=True)
    test_data4.drop(feature, axis=1, inplace=True)

In [54]:
train_data4.head()

,job_offer,employment,no_company,unemployment,population,GDP,i_rate,CLI,CFI,year,...,인천,전남,전북,제주,충남,충북,"사업시설 관리, 사업 지원 및 임대 서비스업","전문, 과학 및 기술 서비스업",정보통신업,제조업
0,60,0,30219,97,10195064,1562.7,2.75,99.14964,101.0,2013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,104,0,109382,202,10195064,1562.7,2.75,99.14964,101.0,2013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,763,4,23384,1230,10195064,1562.7,2.75,99.14964,101.0,2013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,35,1,33649,70,10195064,1562.7,2.75,99.14964,101.0,2013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,64,0,11578,62,10195064,1562.7,2.75,99.14964,101.0,2013,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [55]:
test_data4.head()

,job_offer,employment,no_company,unemployment,population,GDP,i_rate,CLI,CFI,year,...,인천,전남,전북,제주,충남,충북,"사업시설 관리, 사업 지원 및 임대 서비스업","전문, 과학 및 기술 서비스업",정보통신업,제조업
0,35,2,32388,115,9657969,1918.7,0.5,102.0237,95.4,2021,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,34,0,131425,167,9657969,1918.7,0.5,102.0237,95.4,2021,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,633,7,32604,1431,9657969,1918.7,0.5,102.0237,95.4,2021,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,34,1,48282,122,9657969,1918.7,0.5,102.0237,95.4,2021,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,23,1,18711,104,9657969,1918.7,0.5,102.0237,95.4,2021,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [56]:
from pycaret.regression import *
setup_rgs = setup(data=train_data4, target='job_offer', test_data=test_data4, session_id=42, index=False, preprocess=False)

,Description,Value
0,Session id,42
1,Target,job_offer
2,Target type,Regression
3,Original data shape,"(10200, 31)"
4,Transformed data shape,"(10200, 31)"
5,Transformed train set shape,"(8160, 31)"
6,Transformed test set shape,"(2040, 31)"
7,Numeric features,30


In [57]:
top5_4 = compare_models(n_select=5, sort='MSE', turbo=False)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,13:39:11
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Linear Regression


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,6.0057,285.4465,16.3643,0.9038,0.6774,0.7229,0.9190
rf,Random Forest Regressor,6.0093,298.5728,16.7549,0.8981,0.6797,0.7331,1.6120
gbr,Gradient Boosting Regressor,6.5313,308.4807,17.1430,0.8929,0.8049,0.8126,0.4300
catboost,CatBoost Regressor,6.2215,339.0896,17.7997,0.8878,0.7242,0.7237,1.6730
lightgbm,Light Gradient Boosting Machine,6.3697,405.5527,18.9135,0.8714,0.7125,0.7087,37.6760
xgboost,Extreme Gradient Boosting,6.7971,442.4744,19.6207,0.8513,0.7175,0.7421,0.0840
dt,Decision Tree Regressor,7.9065,544.1408,22.3031,0.8250,0.8545,0.9593,0.0330
br,Bayesian Ridge,10.8291,742.2680,25.8530,0.7720,1.1096,1.6162,0.0130
ridge,Ridge Regression,10.8862,742.4051,25.8631,0.7719,1.1147,1.6340,0.0110
lr,Linear Regression,10.8758,743.9127,25.8836,0.7715,1.1173,1.6249,0.0130


In [58]:
top5_4

[ExtraTreesRegressor(n_jobs=-1, random_state=42),
 RandomForestRegressor(n_jobs=-1, random_state=42),
 GradientBoostingRegressor(random_state=42),
 LGBMRegressor(n_jobs=-1, random_state=42)]

In [59]:
tuned_top5_4 = [tune_model(i) for i in top5_4]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,10.3161,827.5208,28.7667,0.8826,0.8945,0.7506
1,7.5798,369.8219,19.2308,0.9211,0.8537,0.7131
2,7.4459,378.8192,19.4633,0.8523,0.9711,0.7499
3,6.1534,173.8228,13.1842,0.9432,1.0190,0.7351
4,5.9411,107.5595,10.3711,0.9574,1.0544,0.7363
5,6.0240,142.9163,11.9548,0.9383,0.9305,0.7194
6,6.4015,317.2041,17.8102,0.8225,0.8774,0.9519
7,5.9915,238.0745,15.4297,0.8997,0.9098,1.1281
8,6.4485,242.4085,15.5695,0.8892,0.9279,1.0964


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,10.1330,951.3993,30.8448,0.8650,0.8883,0.8401
1,7.4015,364.1385,19.0824,0.9223,0.8728,0.8077
2,7.1451,316.0642,17.7782,0.8767,0.9602,0.8358
3,5.9209,189.9409,13.7819,0.9379,0.9849,0.7548
4,5.7881,108.7779,10.4297,0.9569,1.0118,0.7506
5,6.1496,148.7922,12.1980,0.9357,0.8964,0.7196
6,6.2275,310.9890,17.6349,0.8260,0.8672,0.9016
7,5.8159,248.4743,15.7631,0.8953,0.9075,1.0990
8,6.2260,242.4684,15.5714,0.8892,0.9148,1.0341


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,9.6025,1129.6962,33.6110,0.8397,0.7550,0.7671
1,6.9575,369.6952,19.2275,0.9211,0.7316,0.7215
2,6.5233,307.4367,17.5339,0.8801,0.7562,0.7536
3,4.9538,140.4688,11.8520,0.9541,0.7155,0.6514
4,5.2406,118.8339,10.9011,0.9529,0.8157,0.7514
5,6.0538,151.8281,12.3219,0.9344,0.8152,0.8098
6,5.8978,351.6513,18.7524,0.8033,0.6974,0.7676
7,4.7869,283.6019,16.8405,0.8805,0.6619,0.7670
8,5.1909,229.9002,15.1625,0.8949,0.6574,0.7174


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,9.1370,735.7421,27.1246,0.8956,0.8288,0.8939
1,7.7908,366.8810,19.1541,0.9217,0.8946,0.7940
2,6.9603,339.6685,18.4301,0.8675,0.8584,0.7339
3,5.5862,194.2875,13.9387,0.9365,0.7931,0.6104
4,6.2713,176.2970,13.2777,0.9301,0.9508,0.7904
5,7.5345,214.4237,14.6432,0.9074,1.0119,1.1705
6,6.3745,333.6811,18.2669,0.8133,0.7606,0.8991
7,5.5752,316.0010,17.7764,0.8668,0.7457,0.9041
8,6.0964,283.4547,16.8361,0.8705,0.7591,0.8896


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,10.2621,952.4773,30.8622,0.8648,0.8421,0.9202
1,8.1142,395.6755,19.8916,0.9155,0.8937,0.8838
2,7.8881,442.3120,21.0312,0.8275,0.9331,0.9226
3,6.0167,185.4633,13.6185,0.9394,0.8143,0.7601
4,6.0576,148.0561,12.1678,0.9413,0.8456,0.8394
5,6.5655,166.5403,12.9051,0.9280,0.8617,0.8406
6,6.4363,254.5556,15.9548,0.8576,0.7936,0.9814
7,5.8726,281.5514,16.7795,0.8813,0.7364,0.9225
8,6.4624,312.4075,17.6751,0.8572,0.7508,0.9377


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [60]:
tuned_top5_4

[ExtraTreesRegressor(n_jobs=-1, random_state=42),
 RandomForestRegressor(n_jobs=-1, random_state=42),
 GradientBoostingRegressor(random_state=42),
 LGBMRegressor(bagging_fraction=0.8, bagging_freq=3, feature_fraction=1.0,
               learning_rate=0.5, min_child_samples=100, min_split_gain=0.3,
               n_estimators=190, n_jobs=-1, num_leaves=10, random_state=42,
               reg_alpha=0.1, reg_lambda=0.05)]

In [61]:
# 혼합 모델 생성
blender_5_4 = blend_models(tuned_top5_4)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,8.3346,506.8624,22.5136,0.9281,0.7459,0.7963
1,6.6343,348.8583,18.6777,0.9255,0.7069,0.6604
2,6.7253,357.2006,18.8998,0.8607,0.7574,0.7519
3,4.7666,142.5090,11.9377,0.9534,0.6499,0.6231
4,5.0120,103.8315,10.1898,0.9589,0.7519,0.6926
5,5.4196,124.1137,11.1406,0.9464,0.7845,0.7234
6,5.5074,267.7617,16.3634,0.8502,0.6615,0.7217
7,4.7712,225.7444,15.0248,0.9049,0.6353,0.6956
8,5.2308,259.3510,16.1044,0.8815,0.6448,0.6927


In [62]:
result4 = pull()
display(result4)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,8.3346,506.8624,22.5136,0.9281,0.7459,0.7963
1,6.6343,348.8583,18.6777,0.9255,0.7069,0.6604
2,6.7253,357.2006,18.8998,0.8607,0.7574,0.7519
3,4.7666,142.5090,11.9377,0.9534,0.6499,0.6231
4,5.0120,103.8315,10.1898,0.9589,0.7519,0.6926
5,5.4196,124.1137,11.1406,0.9464,0.7845,0.7234
6,5.5074,267.7617,16.3634,0.8502,0.6615,0.7217
7,4.7712,225.7444,15.0248,0.9049,0.6353,0.6956
8,5.2308,259.3510,16.1044,0.8815,0.6448,0.6927


In [63]:
final_model = finalize_model(blender_5_4)
prediction4 = predict_model(final_model, data=test_data4.drop('job_offer', axis=1))
prediction4

[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8


,employment,no_company,unemployment,population,GDP,i_rate,CLI,CFI,year,month,...,전남,전북,제주,충남,충북,"사업시설 관리, 사업 지원 및 임대 서비스업","전문, 과학 및 기술 서비스업",정보통신업,제조업,prediction_label
0,2,32388,115,9657969,1918.699951,0.50,102.023697,95.400002,2021,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,37.376422
1,0,131425,167,9657969,1918.699951,0.50,102.023697,95.400002,2021,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.456971
2,7,32604,1431,9657969,1918.699951,0.50,102.023697,95.400002,2021,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,580.647249
3,1,48282,122,9657969,1918.699951,0.50,102.023697,95.400002,2021,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,38.701737
4,1,18711,104,9657969,1918.699951,0.50,102.023697,95.400002,2021,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,21.363033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2035,0,2134,2,678159,1968.800049,3.25,98.750893,90.199997,2022,12,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.567476
2036,0,7385,6,678159,1968.800049,3.25,98.750893,90.199997,2022,12,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.464605
2037,0,559,23,678159,1968.800049,3.25,98.750893,90.199997,2022,12,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,8.423480
2038,0,1561,4,678159,1968.800049,3.25,98.750893,90.199997,2022,12,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.813228


In [64]:
real_pred4 = pd.DataFrame({'real':test_data4['job_offer'], 'pred':prediction4['prediction_label']})
real_pred4['pred'] = real_pred4['pred'].astype('int')
real_pred4

,real,pred
0,35,37
1,34,43
2,633,580
3,34,38
4,23,21
...,...,...
2035,0,1
2036,0,0
2037,8,8
2038,1,0


In [65]:
real_pred4.loc[real_pred4['real'] != real_pred4['pred']]['pred'].value_counts()

1      186
2      164
3      105
4       82
5       70
      ... 
640      1
111      1
157      1
654      1
184      1
Name: pred, Length: 133, dtype: int64

In [70]:
print('정답률: ', 100-((133/2040)*100), '%', sep='')

정답률: 93.48039215686275%
